# Video data (masked pixel ratio) analysis

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns

palette5 = sns.color_palette(["#888888", "#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette = palette2
display(palette)
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

## Config

In [ ]:
FPS = 30
REC_SEC = 185
REC_SEC_SHORT = 126

OFFSET = 3 # # 3 ( < 3.6) frames, 99.99 ms which is close to 120 ms (200 - 80 ms)

EXP_TOTAL_FRAMES = FPS * REC_SEC + OFFSET
EXP_TOTAL_FRAMES_SHORT = FPS * REC_SEC_SHORT + OFFSET
target_path = "../data/umineko-2024-v8i-yolov8/predicted-data/*.csv"
path_list = sorted(glob.glob(target_path))
print(f"len(path_list): {len(path_list)}")

## Discrepancies between the logging data and video data

### Differences in the number of expected and actual frames

In [ ]:
video_index_list = []
is_audio_played_list = []
frame_diff_list = []
delay_ms_list = []
delay_s_list = []

for i, path in enumerate(path_list):
    df_frames = pd.read_csv(path)
    test_id = os.path.basename(path)[:5]
    session_id = os.path.basename(path).replace(".csv", "")
    if session_id in ["LBP01_S03", "LBP06_S04"]:
    # LBP01_S03: video recording was not stopped -> more than 40 min video
    # LBP06_S04: playback performed but the video recording terminated at 02:24 likely due to low battery
    #               ( -> the last 10 - 20 sec data were not saved, but this does not affect the analysis)
        # frame_diff = 0
        # is_audio_played = False
        # delay_ms = 0
        # delay_s = 0
        continue # skip
    frame_diff = EXP_TOTAL_FRAMES - len(df_frames)
    # print(f"{i} {test_id} {session_id} {frame_diff}")
    if frame_diff > 10 * FPS:
        frame_diff = EXP_TOTAL_FRAMES_SHORT - len(df_frames)
        is_audio_played = False
    else:
        # frame_diff = EXP_TOTAL_FRAMES - len(df_frames)
        is_audio_played = True
    print(f"{i} {test_id} {session_id} {frame_diff}")
    
    delay_ms = frame_diff / 30 * 1000
    delay_s = delay_ms / 1000
    # print(f"frame diff: {frame_diff}")
    # print(f"delay based on frame diff: {delay_ms:.2f} (ms)")
    # print(f"delay based on frame diff: {delay_s:.2f} (s)")
    video_index_list.append(i)
    is_audio_played_list.append(is_audio_played)
    frame_diff_list.append(frame_diff)
    delay_ms_list.append(delay_ms)
    delay_s_list.append(delay_s)

data_dict = {
    'video_id': video_index_list,
    'playback': is_audio_played_list,
    'frame_diff': frame_diff_list,
    'delay_ms': delay_ms_list,
    'delay_s': delay_s_list,
}

df_diff = pd.DataFrame(data_dict)

In [ ]:
display(df_diff.head())

In [ ]:
print(len(df_diff))
_frame_diff = df_diff["frame_diff"]
print( f"median: {np.median(_frame_diff)}" )
print( f"min: {np.min(_frame_diff)}" )
print( f"max: {np.max(_frame_diff)}" )

### Visualization of the delay

In [ ]:
np.random.seed(558)

GRIDSPEC_KW = {'width_ratios': [1, 4], 'wspace': 0.3, 'hspace': 0.5}
fig, axes = plt.subplots(2, 2, figsize=(20, 10), gridspec_kw=GRIDSPEC_KW)
ax_list = list(axes.flatten())
df_p = df_diff

for i, ax in enumerate(ax_list):

    if i == 0:
        # Violinplot
        violin_parts = sns.violinplot(
            y='frame_diff', data=df_p, inner=None,
            linewidth=0.0, color='#333333', saturation=0.5, ax=ax)
        for pc in violin_parts.collections:
            pc.set_edgecolor('black')
            pc.set_facecolor('#333333')
            pc.set_alpha(0.25)

        # Boxplot
        unique_devices = np.array([0])
        device_positions = np.arange(len(unique_devices)) + 0.25
        box_parts = ax.boxplot(
            df_p['frame_diff'],
            positions=device_positions,
            widths=0.12,
            patch_artist=True,
            showfliers=False,
            medianprops=dict(color='black', linewidth=3.0),
            whiskerprops=dict(color='black', linewidth=2.0),
            capprops=dict(color='black', linewidth=2.0),
            boxprops=dict(facecolor='white', color='black', linewidth=2.0),
        )
        ax.set_xticks(np.arange(len(unique_devices)))
        ax.set_xticklabels("")

        # Data point
        sns.stripplot(y='frame_diff', data=df_p, color="#333333", jitter=True, size=9, alpha=0.7, ax=ax)
        
        ax.set_ylabel('Frame Difference', labelpad=10)
        ax.set_yticks(np.arange(0, 150, 30))
        ax.set_ylim(-10, 115)
        ax.grid(which='major', axis='y')

    elif i == 1:
        
        sns.scatterplot(
            x='video_id', y='frame_diff', hue='playback', 
            palette=palette5[:2], data=df_p, ax=ax
        )
        ax.set_xlabel('Video Index', labelpad=10)
        ax.set_ylabel('Frame Difference', labelpad=10)
        ax.set_yticks(np.arange(0, 150, 30))
        ax.set_ylim(-10, 115)
        ax.grid(which='major')
        ax.legend(title='Audio Playback', loc='lower right', ncol=2)

    elif i == 2:
        # Violinplot
        violin_parts = sns.violinplot(
            y='delay_s', data=df_p, inner=None,
            linewidth=0.0, color='#333333', saturation=0.5, ax=ax)
        for pc in violin_parts.collections:
            pc.set_edgecolor('black')
            pc.set_facecolor('#333333')
            pc.set_alpha(0.25)

        # Boxplot
        unique_devices = np.array([0])
        device_positions = np.arange(len(unique_devices)) + 0.25
        box_parts = ax.boxplot(
            df_p['delay_s'],
            positions=device_positions,
            widths=0.12,
            patch_artist=True,
            showfliers=False,
            medianprops=dict(color='black', linewidth=3.0),
            whiskerprops=dict(color='black', linewidth=2.0),
            capprops=dict(color='black', linewidth=2.0),
            boxprops=dict(facecolor='white', color='black', linewidth=2.0),
        )
        ax.set_xticks(np.arange(len(unique_devices)))
        ax.set_xticklabels("")

        # Data point
        sns.stripplot(y='delay_s', data=df_p, color="#333333", jitter=True, size=9, alpha=0.7, ax=ax)
        
        ax.set_ylabel('Delay Time (s)', labelpad=10)
        ax.set_yticks(np.arange(0, 5, 1.0))
        ax.set_ylim(-0.2, 3.9)
        ax.grid(which='major', axis='y')

    elif i == 3:
        sns.scatterplot(
            x='video_id', y='delay_s', hue='playback', 
            palette=palette5[:2], data=df_p, ax=ax
        )
        ax.set_xlabel('Video Index', labelpad=10)
        ax.set_ylabel('Delay Time (s)', labelpad=10)
        ax.set_yticks(np.arange(0, 5, 1.0))
        ax.set_ylim(-0.2, 3.9)
        ax.grid(which='major')
        ax.legend(title='Audio Playback', loc='lower right', ncol=2)

    ax.yaxis.set_minor_locator(tck.AutoMinorLocator(2))
    if i >= 2:
        ax.yaxis.set_major_formatter(tck.FormatStrFormatter('%.1f'))
    ax.grid(axis='both', which='minor', alpha=0.5)

    # Label for plot
    plot_label_list = ['(a)', '(b)', '(c)', '(d)']
    if i == 0 or i == 2:
        ax.text(-0.5, 1.05, f'{plot_label_list[i]}', transform=ax.transAxes, fontsize=30, fontweight='bold')
    else:
        ax.text(-0.125, 1.05, f'{plot_label_list[i]}', transform=ax.transAxes, fontsize=30, fontweight='bold')
    
plt.show()

save_dir = "../output/figure-for-paper/"
# fig.savefig(f"{save_dir}/png/fig_s06_zure_video_frame_exp_actual.png", dpi=350, bbox_inches="tight", pad_inches=0.25, transparent=False)
# fig.savefig(f"{save_dir}/pdf/fig_s06_zure_video_frame_exp_actual.pdf", dpi=600, bbox_inches="tight", pad_inches=0.25, transparent=False)